# Code zu Lanczos-Verfahren mithilfe von QR-Verfahren

In [1]:
import numpy as np
from scipy.stats import unitary_group
from scipy.linalg import block_diag
from scipy.linalg import qr
import time
import matplotlib.pyplot as plt

### Funktionen zum generiern von Matrizen

In [2]:
def gen_hess(n, compl = False):
    if compl:
        a = np.random.rand(n) + np.random.rand(n)*1j
        b = np.random.rand(n-1) + np.random.rand(n-1)*1j
        c = np.random.rand(n-1) + np.random.rand(n-1)*1j
        return np.diag(b, -1) + np.diag(a,0) + np.diag(c, 1)
    else:
        A = np.zeros((n,n))
        a = np.random.rand(n)*50
        A = A + np.diag(a)
        for j in range(n-1):
            b = np.random.rand(n-j-1)*50
            A = A + np.diag(b,j+1)
        return np.diag(np.random.rand(n-1)*50,-1) +  A
    

def gen_upper_tri(eig, compl = False):
    n = eig.shape[0]
    if compl:
        return np.triu(np.random.rand(n,n) + np.random.rand(n,n) * 1j, 1) + np.diag(eig)
    else:
        return np.triu(np.random.rand(n,n),1) + np.diag(eig)
    

def gen_hermite(eig, compl = False):
    n = eig.shape[0]
    B = unitary_group.rvs(n)
    return B.T.conj()@np.diag(eig)@B


def gen_rand_mat(eig, compl = False):
    n = eig.shape[0]
    if compl:
        B = np.random.rand(n,n) + np.random.rand(n,n)*compl*1j
    else:
        B = np.random.rand(n,n)
    return np.linalg.inv(B)@np.diag(eig)@B



#print(gen_hess(5))
#print(gen_upper_tri(np.array([1,2,3]), True))
#print(gen_hermite(np.array([1,2,3,4])))
#print(gen_rand_mat(np.array([1,2,7,4])))



### Verfahren implementiert

In [5]:
def QR_simple(A,tol = 1e-6):
    count = 0
    while abs(A[1,0]) > tol:
        Q,R = np.linalg.qr(A)
        A = R@Q
        count +=1
    return A, sorted(np.diag(A)) ,count



def QR_shift(A,tol=1e-10):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol:
            rho = A[i,i]
            Q,R = np.linalg.qr(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count +=1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count



def QR_shift2(A,tol=1e-12):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol*(abs(A[i-1,i-1])+abs(A[i,i])):
            w = np.linalg.eigvals(A[i-1:i+1,i-1:i+1])
            if abs(w[0] - A[i,i]) < abs(w[1] - A[i,i]):
                rho = w[0]
            else:
                rho = w[1]
            Q,R = np.linalg.qr(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count += 1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count



def QR_hesse_old(A,tol=1e-12):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol*(abs(A[i-1,i-1])+abs(A[i,i])):
            w = np.linalg.eigvals(A[i-1:i+1,i-1:i+1])
            if abs(w[0] - A[i,i]) < abs(w[1] - A[i,i]):
                rho = w[0]
            else:
                rho = w[1]
            Q,R = QR_decomp_hesse_old(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count += 1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count

def QR_hesse(A,tol=1e-14):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol*(abs(A[i-1,i-1])+abs(A[i,i])):
            w = np.linalg.eigvals(A[i-1:i+1,i-1:i+1])
            if abs(w[0] - A[i,i]) < abs(w[1] - A[i,i]):
                rho = w[0]
            else:
                rho = w[1]
            A = Hessenberg_QR_RQ(A-rho*np.identity(n))            #Statt vorimplementierter QR-Zerlegung hier QR-Zerlegung mit Givens-Rotationen
            A += rho*np.identity(n)
            count += 1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count

def QR_decomp_hesse(A):
    n = A.shape[0]
    Q = np.eye(n,n, dtype = complex)
    
    for i in range(0,n-1):
        if abs(A[i,i]) >= abs(A[i+1,i]):
            t = A[i+1,i]/abs(A[i,i])
            root = (1+abs(t)**2)**(1/2)
            c = A[i,i]/(abs(A[i,i])*root)
            s = t/root
        else:
            t = A[i,i]/abs(A[i+1,i])
            root = (1+abs(t)**2)**(1/2)
            s = A[i+1,i]/(abs(A[i+1,i])*root)
            c = t/root
      
        for j in range(n):
            if j < i:
                temp_2 = Q[i,j]
                Q[i,j] = c.conj()*temp_2 + s.conj()*Q[i+1,j]
                Q[i+1,j] = -s*temp_2 + c*Q[i+1,j]
            else:
                temp_1 = A[i,j]
                A[i,j] = c.conj()*temp_1 + s.conj()*A[i+1,j]
                A[i+1,j] = -s*temp_1 + c*A[i+1,j]
                temp_2 = Q[i,j]
                Q[i,j] = c.conj()*temp_2 + s.conj()*Q[i+1,j]
                Q[i+1,j] = -s*temp_2 + c*Q[i+1,j]
    
    return Q.T.conj(), A

def QR_decomp_hesse_old(A):
    n = A.shape[0]
    Q = np.eye(n,n)

    for i in range(0,n-1):
        if abs(A[i,i]) >= abs(A[i+1,i]):
            t = A[i+1,i]/abs(A[i,i])
            root = (1+abs(t)**2)**(1/2)
            c = A[i,i]/(abs(A[i,i])*root)
            s = t/root
        else:
            t = A[i,i]/abs(A[i+1,i])
            root = (1+abs(t)**2)**(1/2)
            s = A[i+1,i]/(abs(A[i+1,i])*root)
            c = t/root
            
        M = np.array([[c.conj(), s.conj()], [-s, c]])
        G = block_diag(np.eye(i,i), M, np.eye(n-i-2, n-i-2))
        Q = G@Q
        for j in range(i, n):
            temp_1 = A[i,j]
            A[i,j] = c.conj()*temp_1 + s.conj()*A[i+1,j]
            A[i+1,j] = -s*temp_1 + c*A[i+1,j]
    return Q.T.conj(), A


def Hessenberg_QR_RQ(A):
    n = A.shape[0]
    c_arr = np.zeros(n, dtype = complex)
    s_arr = np.zeros(n, dtype = complex)
    for i in range(0,n-1):
        if abs(A[i,i]) >= abs(A[i+1,i]):                     
            t = A[i+1,i]/abs(A[i,i])                         
            root = (1+abs(t)**2)**(1/2)                     
            c_arr[i] = A[i,i]/(abs(A[i,i])*root)                  
            s_arr[i] = t/root          
        else:
            t = A[i,i]/abs(A[i+1,i])                 
            root = (1+abs(t)**2)**(1/2)                      
            s_arr[i] = A[i+1,i]/(abs(A[i+1,i])*root)               
            c_arr[i] = t/root                              
            
        for j in range(i,n):
                temp_1 = A[i,j]
                A[i,j] = c_arr[i].conj()*temp_1 + s_arr[i].conj()*A[i+1,j] 
                A[i+1,j] = -s_arr[i]*temp_1 + c_arr[i]*A[i+1,j]
                
    for i in range(0,n-1):
        for j in range(0,i+2):
            temp_1 = A[j,i]
            A[j,i] = c_arr[i]*temp_1 + s_arr[i]*A[j,i+1]
            A[j,i+1] = -s_arr[i].conj()*temp_1 + c_arr[i].conj()*A[j,i+1]
    
    return A


def lanczos(A,k = 0):
    n = A.shape[1]
    if k == 0:
        k = n
        
    v0 = np.random.rand(n)
    v = [v0/np.linalg.norm(v0)]
    gam = [v[0].T.conj()@A@v[0]]
    w = (A - gam[0]*np.identity(n))@v[0]
    delta = [np.linalg.norm(w)]
    i = 0
    while delta[i] > 1e-10 and i < k-1:
        v.append(w/delta[i])
        i +=1
        gam.append(v[i].T.conj()@A@v[i])
        w = (A - gam[i]*np.identity(n))@v[i] - delta[i-1]*v[i-1]
        delta.append(np.linalg.norm(w))
    T = np.diag(delta[:-1], -1) + np.diag(gam) + np.diag(delta[:-1], 1)
    return QR_hesse(T)

def arnoldi(A, dim, k = 0):
    n = dim
    if k == 0:
        k = n
        
    v0 = np.random.rand(n)
    v = [v0/np.linalg.norm(v0)]
    h = np.zeros((k,k))
    
    for j in range(k):
        w = A(v[j])
        for l in range(j+1):
            h[l][j] = w.T.conj()@v[l]
            w = w - h[l][j]*v[l]
        if j < k-1:
            h[j+1][j] = np.sqrt(w.T.conj()@w)
            if abs(h[j+1][j]) < 1e-2:
                return QR_hesse(h[:j+1,:j+1])
            else:
                v.append(w/h[j+1][j])
    #print(sorted(np.linalg.eigvals(h)))
    return QR_hesse(h)

### Testing

In [8]:
def error(eig_1, eig_2):
    eig_1 = np.sort(eig_1)
    eig_2 = np.sort(eig_2)
    return np.linalg.norm(eig_1 - eig_2)


def runtime(f,A):
    start = time.time()
    f(A)
    end = time.time()
    return end - start

def runtime_error(f,A):
    start = time.time()
    S = f(A)
    end = time.time()
    return end - start, S

def runtime_error_lanczos(f,A,k):
    start = time.time()
    S = f(A,k)
    end = time.time()
    return end - start, S

def testing(f, n_1, n_2):
    n_array = np.array([i for i in range(n_1,n_2)])
    time_array = []
    error_array = []
    for n in n_array:
        eigv = np.sort(50*np.random.rand(n)-25)
        A = gen_hermite(eigv)
        t, A = runtime_error(f,A)
        err = error(A[1], eigv)
        time_array.append(t)
        error_array.append(err)
    plt.plot(n_array, time_array, label = "runtime")
    #plt.plot(n_array, error_array, 'x', label = "errors")
    plt.legend()
    plt.show()

    
def testing_lanczos(f,k, n_1, n_2):
    n_array = np.array([i for i in range(n_1,n_2)])
    time_array = []
    error_array = []
    for n in n_array:
        eigv = np.sort(50*np.random.rand(n)-25)
        A = gen_hermite(eigv)
        t, A = runtime_error_lanczos(f,A,k)
        err = error(A[1][0], eigv[0])
        time_array.append(t)
        error_array.append(err)
    plt.plot(n_array, time_array, label = "runtime lanczos")
    #plt.plot(n_array, error_array, 'x', label = "errors")
    plt.legend()
    plt.show()
    
def selective_error(eig_1, eig_2, k = 1):
    n_1 = len(eig_1)
    n_2 = len(eig_2)
    error_1 = np.linalg.norm(eig_1[:k] - eig_2[:k])
    error_2 = np.linalg.norm(eig_1[:n_1-k-1:-1] - eig_2[:n_2-k-1:-1])
    return error_1 + error_2

# ------- testing starts here -------

n = 1000

mu, sigma = 3, 1
eigv = np.sort(np.random.rand(n)*4000)

#eigv = np.array([-(i+1)*10000 for i in range(n)])
A = gen_rand_mat(eigv)

def matmul(v):
    return A@v


print(np.sort(eigv))
#print(np.round(QR_simple(A)[1],3))
#print(np.round(QR_shift(A)[1],3))
#print(np.round(QR_shift2(A)[1],3))
#print(np.round(QR_hesse(A)[0],3))
#print(np.round(lanczos(A,30)[1],3))
#print(np.linalg.eigvals(A))
#print(np.sort(np.round(arnoldi(matmul,n)[1], 3)))

#print(error(QR_simple(A)[1], eigv))
#print(error(QR_shift(A)[1], eigv))
#print(error(QR_shift2(A)[1], eigv))
#print(error(lanczos(A)[1], eigv))

#print(error(arnoldi(lambda x: A@x,n, 50)[1],eigv))
#print(arnoldi(lambda x: A@x,n, 50)[1])
#print(runtime(QR_simple, A))
#print(runtime(QR_shift, A))
#print(runtime(QR_shift2, A))
#print(runtime_error_lanczos(lanczos, A, 10)[0])


[  12.23478958   26.08135842   26.58857535   28.92226697   35.56597393
   41.18667124   41.81082426   44.66498781   48.46460494   49.57878527
   51.56467081   61.16396311   63.55568208   65.52726095   65.79087995
   68.66610027   70.33518352   75.83452364   81.94961264   82.82013851
   90.46262727   98.81882326  101.81839891  104.32330239  104.7714025
  105.77231152  106.36946985  110.96779077  115.21409135  116.36126416
  118.46204427  119.61173036  120.60715544  120.68336655  123.53950499
  125.28415766  126.1108124   127.540863    131.25968142  137.54804896
  139.87385289  143.42319188  143.98554855  157.25277659  158.93990737
  159.65780343  162.90551272  163.70606165  167.85471153  168.94068066
  170.461123    178.15524731  183.53338831  185.34495578  186.65906371
  187.33899293  190.84622921  196.08276479  196.33508023  211.42261467
  212.41159208  213.08824208  222.75566241  228.26437634  230.68837985
  231.10751301  231.27918426  237.27412551  247.50774043  247.87176184
  252.6

In [9]:
k = 120
for k in range(5,50):
    #lanczos_test = lanczos(A,k)[1]
    #print(np.sort(lanczos_test))
    #err_ = error(lanczos_test[:len(lanczos_test)-4:-1], eigv[:len(eigv)-4:-1])


   # print("lanczos = ", err_)

    arnoldi_test = arnoldi(lambda x: A@x,n,k)[1]
    #print(np.sort(arnoldi_test))
    error_1 = abs(arnoldi_test[-1]-eigv[-1])

    print("arnoldi=", error_1)

c:\users\salli\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:150: ComplexWarning: Casting complex values to real discards the imaginary part
c:\users\salli\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:151: ComplexWarning: Casting complex values to real discards the imaginary part
c:\users\salli\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:156: ComplexWarning: Casting complex values to real discards the imaginary part
c:\users\salli\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:157: ComplexWarning: Casting complex values to real discards the imaginary part


arnoldi= 371.8661912400612
arnoldi= 24.783569915169664
arnoldi= 428.29448432432486
arnoldi= 286.3668848775102
arnoldi= 205.8119846433383
arnoldi= 201.31850466114258
arnoldi= 66.78828409822654
arnoldi= 31.773349987037363
arnoldi= 91.7290960608822
arnoldi= 17.402408918785568
arnoldi= 6.51450968480458
arnoldi= 8.659712552869223
arnoldi= 54.27697898875485
arnoldi= 35.28491513988134
arnoldi= 1.3318725844246728
arnoldi= 4.287335850206546
arnoldi= 10.617851402794713
arnoldi= 6.835213450924584
arnoldi= 13.602781914180468
arnoldi= 3.7584285819293655
arnoldi= 9.043762451111434
arnoldi= 9.30332055692179
arnoldi= 6.92833322784918
arnoldi= 3.4946235731626984
arnoldi= 1.435636905507181
arnoldi= 0.8254428631348674
arnoldi= 0.8312456799139909
arnoldi= 8.05799361870504
arnoldi= 11.589301923160003
arnoldi= 8.28862043014888
arnoldi= 3.8547273628955736
arnoldi= 1.2065866469206412


KeyboardInterrupt: 

In [ ]:
k = [i*10 for i in range(1,60)]

n = len(k)
mat_array = [0]*n
mat_array_2 = [0]*n
for j in range(n):
    mat_array[j] = (gen_hess(k[j]))
    mat_array_2[j] = gen_rand_mat(np.random.rand(k[j])*500)
time_good = [0]*n
time_bad = [0]*n


In [ ]:
for j in range(len(k)):
    time_good[j] = runtime(QR_decomp_hesse, mat_array[j])
    time_bad[j] = runtime(np.linalg.qr, mat_array_2[j])

In [ ]:
%matplotlib notebook

plt.plot(k, time_good, '-o', label = "good")
plt.plot(k, time_bad, '-o',label = "bad")

plt.legend()
plt.show()

In [ ]:
n_dim_2 = 500 

eigvalues_2 = np.sort(2*np.random.rand(n_dim))
eigvalues_2[-1] = 3
eigvalues_2[-2] = 2.5
eigvalues_2[0] = -1
eigvalues_2[1] = -0.5
L = gen_hermite(eigvalues_2)

j = 10

In [ ]:
lanczos_arr = [[0]]*j

for l in range(j):
    lanczos_arr[l] = lanczos(L,l+2)[1]

In [ ]:
%matplotlib notebook

for l in range(n_dim):
    plt.plot(12,eigvalues[l], 'x')
for l in range(len(lanczos_arr)):
    for k in range(len(lanczos_arr[l])):
        plt.plot(l+2, lanczos_arr[l][k], 'x')
        
plt.axvline(x=11.5)
plt.xlabel("Kyrolvraumdimension, bei 12 Eigenwerte")
plt.ylabel("Wertebereich der Eigenwerte")
plt.show()